In [22]:
from PIL import Image
import h5py
import anubisPlotUtils as anPlot
import json
import numpy as np
import os
import hist as hi
import matplotlib
import matplotlib.pyplot as plt
from itertools import combinations
import matplotlib.colors as colors
matplotlib.use('TkAgg')  # or 'Qt5Agg', 'GTK3Agg', etc.
import mplhep as hep
hep.style.use([hep.style.ATLAS])
import sys
import AnalysisToolAnubis as AT
import pandas as pd
import tkinter as tk
from tkinter import ttk
from pandasgui import show
from matplotlib.patches import Circle
from matplotlib.backends.backend_pdf import PdfPages
from scipy.signal import convolve2d
from scipy.ndimage import label, find_objects
from scipy.spatial.distance import pdist, squareform
from matplotlib.backends.backend_pdf import PdfPages
import ipywidgets as widgets
from IPython.display import display
import sys
from matplotlib.ticker import MaxNLocator



In [2]:
#Importing the data
thisData = AT.importDatafile('C:\\Users\\Peter\\OneDrive - University of Cambridge\\Desktop\\Project Excel Work\\PartIIIRPC\\ProAnubis_CERN\\ProAnubisData\\60sRun_24_3_4.h5')


In [3]:
mapping = {
    0: {
        (0, 31): ('rpc1', 'eta'),
        (32, 63): ('rpc1', 'phi1'),
        (64, 95): ('rpc1', 'phi2'),
        (96, 127): ('rpc2', 'eta'),
    },
    1: {
        (0, 31): ('rpc2', 'phi1'),
        (32, 63): ('rpc2', 'phi2'),
        (64, 95): ('rpc3', 'eta'),
        (96, 127): ('rpc3', 'phi1'),
    },
    2: {
        (0, 31): ('rpc3', 'phi2'),
        (32, 63): ('rpc4', 'eta'),
        (64, 95): ('rpc4', 'phi1'),
        (96, 127): ('rpc4', 'phi2'),
    },
    3: {
        (0, 31): ('rpc5', 'eta'),
        (32, 63): ('rpc5', 'phi1'),
        (64, 95): ('rpc5', 'phi2'),
        (96, 127): ('rpc6', 'eta'),
    },
    4: {
        (0, 31): ('rpc6', 'phi1'),
        (32, 63): ('rpc6', 'phi2'),
    },
}

In [4]:
def assign_to_coordinates(tdc, channel, mapping):
    for channel_range, (rpc, side) in mapping.get(tdc, {}).items():
        if channel_range[0] <= channel <= channel_range[1]:
            offset = channel - channel_range[0]
            if side == 'eta':
                return [rpc, 'eta', [0, offset]]
            elif side in 'phi1':
                return [rpc, 'phi', [offset, 0]]
            elif side in 'phi2':
                return[rpc, 'phi', [offset + 32, 0]]
            
    return ['Null', 'Null', [tdc, channel]]  # If no match found
def remake_data(Data, mapping):
    full_data = []
    for tdc in range(0, len(Data)):
        for events in range(0, len(Data[tdc])): 
            for hits in Data[tdc][events]:
                channel = (hits>>24)&0x7f
                coord = assign_to_coordinates(tdc, channel, mapping)
                event_time = hits&0xfffff
                
            
                
                full_data.append({
                    'tdc': tdc,
                    'event number': events,
                    'channel':channel,
                    'rpc number': coord[0],
                    'strip direction': coord[1],
                    'hit coordinate': coord[2],
                    'time': event_time
                })
    df = pd.DataFrame(full_data)
        
    return df
    
    

In [5]:
df = remake_data(thisData, mapping)


In [27]:
show(df)

PandasGUI INFO — pandasgui.gui — Opening PandasGUI
INFO:pandasgui.gui:Opening PandasGUI


In [6]:
#slightly different way to store data, but does the same thing, it goes like rpc - event - strip
etaHits, phiHits = AT.divideHitCountsByRPC(thisData)

In [7]:
def calculate_cluster_metrics(event, minHit=0, maxHit=20):
    cluster_sizes = []
    cluster_locations = []
    current_cluster_size = 0
    start_index = None
    
    for index, i in enumerate(event):
        if i != 0:
            current_cluster_size += i
            if start_index is None:
                start_index = index  
        elif i == 0 and current_cluster_size >= minHit and current_cluster_size < maxHit:
            if start_index is not None:           
                cluster_midpoint = (start_index + index) / 2.0 
                cluster_locations.append(cluster_midpoint)
                cluster_sizes.append(current_cluster_size)
                current_cluster_size = 0
                start_index = None 
        elif i == 0:
            current_cluster_size = 0
            start_index = None 
            
    if current_cluster_size >= minHit and current_cluster_size < maxHit and start_index is not None:
        cluster_midpoint = (start_index + len(event)) / 2.0 
        cluster_locations.append(cluster_midpoint)
        cluster_sizes.append(current_cluster_size)
    
    return cluster_locations, cluster_sizes


In [8]:
columns = ['RPC Number', 'Event Number', 'cluster_locations', 'Cluster_size']
dfcluseta = pd.DataFrame(columns=columns)
dfclusphi = pd.DataFrame(columns=columns)

for i, rpc_data in enumerate(etaHits):
    for j, event in enumerate(rpc_data):
        cluster_locations, cluster_sizes = calculate_cluster_metrics(event)
        for _ in range(len(cluster_sizes)):
            
            dfcluseta = dfcluseta.append({'RPC Number': int(i), 'Event Number': int(j), 'cluster_locations': cluster_locations[_], 'Cluster_size': int(cluster_sizes[_])}, ignore_index=True)

for i, rpc_data in enumerate(phiHits):
    for j, event in enumerate(rpc_data):
        cluster_locations, cluster_sizes = calculate_cluster_metrics(event)
        filtered_pairs = [(x, y) for x, y in zip(cluster_locations, cluster_sizes) if x >= 1]

        if filtered_pairs:
            filtered_my_list, filtered_list2 = zip(*filtered_pairs)

            cluster_locations = list(filtered_my_list)
            cluster_sizes = list(filtered_list2)
        else:

            cluster_locations = []
            cluster_sizes = []
        for _ in range(len(cluster_sizes)):
            
            dfclusphi = dfclusphi.append({'RPC Number': int(i), 'Event Number': int(j), 'cluster_locations': cluster_locations[_], 'Cluster_size': int(cluster_sizes[_])}, ignore_index=True)

C:\Users\Peter\AppData\Local\Temp\ipykernel_33908\695374553.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dfcluseta = dfcluseta.append({'RPC Number': int(i), 'Event Number': int(j), 'cluster_locations': cluster_locations[_], 'Cluster_size': int(cluster_sizes[_])}, ignore_index=True)
C:\Users\Peter\AppData\Local\Temp\ipykernel_33908\695374553.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dfcluseta = dfcluseta.append({'RPC Number': int(i), 'Event Number': int(j), 'cluster_locations': cluster_locations[_], 'Cluster_size': int(cluster_sizes[_])}, ignore_index=True)
C:\Users\Peter\AppData\Local\Temp\ipykernel_33908\695374553.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dfcluseta = dfcluseta.append({'RPC Num

In [45]:
print(calculate_cluster_metrics(phiHits[0][0]))

([52.5], [5])


In [9]:
rpc_dfseta = {rpc: dfcluseta[dfcluseta['RPC Number'] == rpc].reset_index(drop=True) for rpc in dfcluseta['RPC Number'].unique()}
rpc_dfsphi = {rpc: dfclusphi[dfclusphi['RPC Number'] == rpc].reset_index(drop=True) for rpc in dfclusphi['RPC Number'].unique()}

def plot_rpc_histograms(rpc_dfseta, rpc_dfsphi):
    with PdfPages('C:\\Users\\Peter\\OneDrive - University of Cambridge\\Desktop\\Project Excel Work\\PartIIIRPC\\ProAnubis_CERN\\ProAnubis Plots\\rpc_cluster_analysisaLL.pdf') as pdf:
        for rpc in rpc_dfseta.keys():
            df_eta = rpc_dfseta[rpc]
            df_phi = rpc_dfsphi[rpc]
            for df, dataset_name in zip([df_eta, df_phi], ['eta', 'phi']):
                all_cluster_locations = []
                all_cluster_sizes = []
                
                for _, row in df.iterrows():
                    locs = row['cluster_locations']
                    sizes = row['Cluster_size']
                    
                    if not isinstance(locs, list):
                        locs = [locs] if not pd.isnull(locs) else []
                    if not isinstance(sizes, list):
                        sizes = [sizes] if not pd.isnull(sizes) else []
                    
                    all_cluster_locations.extend(locs)
                    all_cluster_sizes.extend(sizes)
                
                locations_series = pd.Series(all_cluster_locations)
                sizes_series = pd.Series(all_cluster_sizes)
                
                location_frequencies = locations_series.value_counts().sort_index()
                size_frequencies = sizes_series.value_counts().sort_index()
                
                fig, axs = plt.subplots(1, 2, figsize=(12, 6))
                fig.suptitle(f'RPC {rpc} - {dataset_name.upper()}')

                if not location_frequencies.empty:
                    axs[0].bar(location_frequencies.index, location_frequencies.values, color='skyblue', edgecolor='black')
                axs[0].set_title(f'Cluster Locations')
                axs[0].set_xlabel('Cluster Location')
                axs[0].set_ylabel('Frequency')

                if not size_frequencies.empty:
                    axs[1].bar(size_frequencies.index, size_frequencies.values, color='salmon', edgecolor='black')
                axs[1].set_title(f'Cluster Sizes')
                axs[1].set_xlabel('Cluster Size')
                axs[1].set_ylabel('Frequency')
                
                if dataset_name == 'phi':
                    axs[0].set_xlim(0, 64)
                    # axs[1].set_xlim(0, 64)
                if dataset_name == 'eta':
                    axs[0].set_xlim(0, 32)
                    # axs[1].set_xlim(0, 32)
                    
                plt.tight_layout(rect=[0, 0.03, 1, 0.95])
                pdf.savefig(fig)
                plt.close()


In [56]:
plot_rpc_histograms(rpc_dfseta, rpc_dfsphi)

In [11]:
etaTime, phiTime = AT.divideEventTimesByRPC(thisData)

In [10]:
show(df)

PandasGUI INFO — pandasgui.gui — Opening PandasGUI
INFO:pandasgui.gui:Opening PandasGUI


It would be easier to work straight from the data frame
first, we want to test how does dark counts work, by only choosing hits that are larger than 300ns

In [9]:
df_filtered300plus = df[df['time'] <= 245]

In [8]:
def dataframe_to_thisData(df):
    max_tdc = df['tdc'].max()
    max_event_number = df['event number'].max()
    
    reconstructed_data = [[[] for _ in range(max_event_number + 1)] for _ in range(max_tdc + 1)]

    for index, row in df.iterrows():
        tdc = row['tdc']
        event_number = row['event number']
        channel = row['channel']
        time = row['time']
        hits = (channel << 24) | time
        reconstructed_data[tdc][event_number].append(hits)

    return reconstructed_data

There wasn't a clear cutoff, but it is clear that below 240 and above 300 we are getting a LOT of single hits, these are ofcourse dark hits.
we should really ignore anything above 10, since that probably means when all the strips fired off at the same time

We can arbitrarily define a timing window of lets say 20 nanoseconds for each event to be considered independent to each other, this will allow me to generate a series of coincidence, and actually finding the coincidences correctly rather than arbitrarily...

In [12]:
show(df)

PandasGUI INFO — pandasgui.gui — Opening PandasGUI
INFO:pandasgui.gui:Opening PandasGUI


In [28]:
df = df.sort_values(by=['event number', 'rpc number', 'time'])

In [11]:
def analyze_clusters(df, window_size=5):
    all_clusters = []
    PHI_POINTS = 64
    ETA_POINTS = 32

    for event_number, event_group in df.groupby('event number'):
        for rpc, rpc_group in event_group.groupby('rpc number'):
            for direction, direction_group in rpc_group.groupby('strip direction'):
                direction_group = direction_group.copy()
                direction_group['sort_key'] = direction_group['hit coordinate'].apply(lambda coord: coord[0] if direction == 'phi' else coord[1])
                sorted_direction_group = direction_group.sort_values(by=['sort_key', 'time'])

                clusters = []
                current_cluster = []
                last_hit_time = -1

                for _, hit in sorted_direction_group.iterrows():
                    hit_point = hit['sort_key']
                    hit_time = hit['time']

                    # Start a new cluster or continue the current one
                    if not current_cluster or (
                        (hit_point - current_cluster[-1][0] <= 1 or
                        (direction == 'phi' and abs(hit_point - current_cluster[-1][0]) == PHI_POINTS - 1) or
                        (direction == 'eta' and abs(hit_point - current_cluster[-1][0]) == ETA_POINTS - 1))
                        and hit_time - last_hit_time <= window_size
                    ):
                        current_cluster.append((hit_point, hit_time))
                    else:
                        if current_cluster:
                            clusters.append(process_cluster(current_cluster))
                        current_cluster = [(hit_point, hit_time)]

                    last_hit_time = hit_time

                # Process the last cluster
                if current_cluster:
                    clusters.append(process_cluster(current_cluster))

                # Flatten the clusters into the all_clusters list
                for cluster in clusters:
                    location = (cluster['start'] + cluster['end']) / 2
                    size = cluster['size']
                    event_time = cluster['event_time']
                    all_clusters.append([event_number, rpc, direction, location, size, event_time])

    cluster_df = pd.DataFrame(all_clusters, columns=['Event Number', 'RPC Number', 'Direction', 'Cluster Location', 'Cluster Size', 'Cluster Event Time'])
    return cluster_df

def process_cluster(cluster_points):
    sorted_points = sorted(cluster_points, key=lambda x: x[0])
    start = sorted_points[0][0]
    end = sorted_points[-1][0]
    size = len(sorted_points)
    # Calculate the average event time for the cluster
    event_time = sum(point[1] for point in sorted_points) / size
    return {'start': start, 'end': end, 'size': size, 'event_time': event_time}

First, partitioning the dataset into subsets based on event number and rpc number, ensures that clusters are identified within the specific context of each event and RPC. 
Within each stratified subset, the data further segregated by strip direction(either phi or eta). By treating these directions separately, the function respects the inherent spatial distinction. 
Sequential evaluation with dynamic clustering, As the functino iterates through the sorted hits of each directionally and contextually isolated subset, it employs a dynamic clustering mechanism that builds clusters based on btoth spatial adjacency and temporal proximity. This process is sensitive to the continuity of datra points, allowing for the formation of clusters that are not only adjacent in space but also occur within a narrowly defined time window. 
The incorporation of a temporal window specified by window size introduces a nuanced layer of temporal analysis, ensuring that only hits occuring within a close temporal successino are considered part of the same lcuster. This aspect is particularly relevant in scenarios where the temporal dimension provides critical insight... probably

after dynamically assembling lcusters, the functino computes the characteristics of each cluster. 

In [12]:
Analyzeddf = analyze_clusters(df)

In [48]:
#remove noise
Analyzeddf = Analyzeddf.loc[Analyzeddf['Cluster Size'] <= 15]
Analyzeddf = Analyzeddf.loc[Analyzeddf['Cluster Location'] != 0]

In [44]:
show(Analyzeddf)

PandasGUI INFO — pandasgui.gui — Opening PandasGUI
INFO:pandasgui.gui:Opening PandasGUI


In [7]:
show(df)

PandasGUI INFO — pandasgui.gui — Opening PandasGUI
INFO:pandasgui.gui:Opening PandasGUI


In [23]:
def print_progress_bar(iteration, total, prefix='', suffix='', length=50, fill='█', printEnd="\r"):
    percent = ("{0:.1f}").format(100 * (iteration / float(total)))
    filledLength = int(length * iteration // total)
    bar = fill * filledLength + '-' * (length - filledLength)
    print(f'\r{prefix} |{bar}| {percent}% {suffix}', end=printEnd)
    if iteration == total: 
        print()

If the RPC dimensions are 1 X 1.6, meaning strip separation in eta is 0.03125, and in phi is 0.025, so need to message the data.

In [53]:
def analyze_inter_rpc_linearity_with_timing(df):
    rpc_info = {
        'rpc1': {'position': 0, 'next_window': 5},
        'rpc2': {'position': 0.1, 'next_window': 5},
        'rpc3': {'position': 0.2, 'next_window': 10},
        'rpc4': {'position': 60.2, 'next_window': 15},
        'rpc5': {'position': 120.2, 'next_window': 5},
        'rpc6': {'position': 120.3, 'next_window': None}
    }

    paths = []
    total_events = df['Event Number'].nunique()
    event_counter = 0

    for event_number, event_group in df.groupby('Event Number'):
        event_counter += 1
        print_progress_bar(event_counter, total_events, prefix='Progress:', suffix='Complete', length=50)

        for direction in ['eta', 'phi']:
            direction_group = event_group[event_group['Direction'] == direction]
            all_clusters = []
            
            for rpc, rpc_data in rpc_info.items():
                rpc_group = direction_group[direction_group['RPC Number'] == rpc]
                for _, row in rpc_group.iterrows():
                    location_scaling = 3.125 if direction == 'eta' else 2.5
                    cluster_size_scaled = max(row['Cluster Size'] * location_scaling, location_scaling) / 2 
                    all_clusters.append({
                        'rpc': rpc,
                        'position': rpc_data['position'],
                        'location': row['Cluster Location'] * location_scaling,
                        'event_time': row['Cluster Event Time'],
                        'uncertainty': cluster_size_scaled,
                        'original_location': row['Cluster Location']  
                    })

            combination_metrics = []
            for combo in combinations(all_clusters, 3):
                if len(set(c['rpc'] for c in combo)) == 3: 
                    positions = np.array([c['position'] for c in combo])
                    locations = np.array([c['location'] for c in combo])
                    uncertainties = np.array([c['uncertainty'] for c in combo])
                    try:
                        slope, intercept = np.polyfit(positions, locations, 1)
                        rss = np.sum(((slope * positions + intercept) - locations) ** 2 / uncertainties ** 2)
                        time_window = max(c['event_time'] for c in combo) - min(c['event_time'] for c in combo)
                        combination_metrics.append((time_window, rss, event_number, direction, slope, intercept, combo))
                    except np.linalg.LinAlgError:
                        continue

            best_combinations = sorted(combination_metrics, key=lambda x: (x[0], x[1])) 
            if best_combinations:
                selected_combination = best_combinations[0]  
                paths.append({
                    'Event Number': selected_combination[2], 
                    'Direction': selected_combination[3], 
                    'Slope': selected_combination[4], 
                    'Intercept': selected_combination[5],
                    'Used Coordinates': [(c['rpc'], c['original_location'], c['uncertainty']) for c in selected_combination[6]]
                })

    path_df = pd.DataFrame(paths)
    return path_df

In [54]:
temporarydf = analyze_inter_rpc_linearity_with_timing(Analyzeddf)

Progress: |██████████████████████████████████████████████████| 100.0% Complete


In [55]:
show(temporarydf)

PandasGUI INFO — pandasgui.gui — Opening PandasGUI
INFO:pandasgui.gui:Opening PandasGUI


In [56]:
show(Analyzeddf)

PandasGUI INFO — pandasgui.gui — Opening PandasGUI
INFO:pandasgui.gui:Opening PandasGUI


Roughly 166 muons per second at sea level, if it is at sea level and flat, we should expect 10,000 muons, We have only observed 576 muons, does this make sense?

In [43]:
rpc_info = {
    'rpc1': {'position': 0, 'next_window': 5},
    'rpc2': {'position': 0.1, 'next_window': 5},
    'rpc3': {'position': 0.2, 'next_window': 10},
    'rpc4': {'position': 6.2, 'next_window': 15},
    'rpc5': {'position': 12.2, 'next_window': 5},
    'rpc6': {'position': 12.3, 'next_window': None}
}

# Plot RPC positions
for rpc, info in rpc_info.items():
    plt.plot([0, 100], [info['position'], info['position']], label=rpc)

# Example trajectory data (for demonstration purposes)
# Assuming this data includes slope and intercept for a trajectory line
trajectories = [
    {'event_number': 1, 'direction': 'eta', 'slope': -6.250000000000139, 'intercept': 128.54166666666669},
    # Add more trajectories as needed
]

# Overlay trajectory lines
x_values = np.linspace(0, 100, 1000) # Adjust these x values as per your data's actual range
for traj in trajectories:
    y_values = traj['slope'] * x_values + traj['intercept']
    plt.plot(x_values, y_values, '--', label=f"Trajectory {traj['event_number']}, {traj['direction']}")

plt.xlabel('X Position')
plt.ylabel('RPC Height')
plt.title('RPC Heights and Trajectory Lines')
plt.legend()
plt.show()

In [33]:
def plot_rpc_histograms_New(dataframe, output_file):
    with PdfPages(output_file) as pdf:
        for (rpc_number, direction), group in dataframe.groupby(['RPC Number', 'Direction']):
            fig, axs = plt.subplots(1, 3, figsize=(18, 6))
            fig.suptitle(f'RPC {rpc_number} - Direction {direction.upper()}')
            

            location_bins = np.arange(group['Cluster Location'].min(), group['Cluster Location'].max() + 2) - 0.5
            axs[0].hist(group['Cluster Location'].dropna(), bins=location_bins, color='skyblue', edgecolor='black')
            axs[0].set_title('Cluster Locations')
            axs[0].set_xlabel('Cluster Location')
            axs[0].set_ylabel('Frequency')
            axs[0].xaxis.set_major_locator(MaxNLocator(integer=True))
            

            size_bins = np.arange(group['Cluster Size'].min(), group['Cluster Size'].max() + 2) - 0.5
            axs[1].hist(group['Cluster Size'].dropna(), bins=size_bins, color='salmon', edgecolor='black')
            axs[1].set_title('Cluster Sizes')
            axs[1].set_xlabel('Cluster Size')
            axs[1].set_ylabel('Frequency')
            axs[1].xaxis.set_major_locator(MaxNLocator(integer=True))


            event_time_counts = group['Cluster Event Time'].value_counts().sort_index()
            axs[2].vlines(event_time_counts.index, 0, event_time_counts.values, colors='lightgreen', linestyles='-', lw=2)
            axs[2].set_title('Cluster Event Times')
            axs[2].set_xlabel('Cluster Event Time')
            axs[2].set_ylabel('Frequency')
            axs[2].set_ylim(bottom=0)
            axs[2].xaxis.set_major_locator(MaxNLocator(integer=True))
            
            plt.tight_layout(rect=[0, 0.03, 1, 0.95])
            pdf.savefig(fig)
            plt.close()

In [34]:
plot_rpc_histograms_New(Analyzeddf, 'C:\\Users\\Peter\\OneDrive - University of Cambridge\\Desktop\\Project Excel Work\\PartIIIRPC\\ProAnubis_CERN\\ProAnubis Plots\\1D timing plot\\plot.pdf')

In [39]:
dfDark = df.loc[df['time'] >=330]
AnalyzeddfDark = analyze_clusters(dfDark)
AnalyzeddfDark = AnalyzeddfDark.loc[AnalyzeddfDark['Cluster Size'] <= 15]
plot_rpc_histograms_New(AnalyzeddfDark, 'C:\\Users\\Peter\\OneDrive - University of Cambridge\\Desktop\\Project Excel Work\\PartIIIRPC\\ProAnubis_CERN\\ProAnubis Plots\\1D timing plot\\plotDark.pdf')

In [37]:
show(dfDark)

PandasGUI INFO — pandasgui.gui — Opening PandasGUI
INFO:pandasgui.gui:Opening PandasGUI


In [135]:
def find_coincidences(group):
    coincidences = []
    for i in range(len(group)):
        for j in range(i + 1, len(group)):

            time_i = group.iloc[i]['time']
            time_j = group.iloc[j]['time']
            coord_i = group.iloc[i]['hit coordinate']
            coord_j = group.iloc[j]['hit coordinate']
            

            if (coord_i[0] != 0 and coord_j[1] != 0 and coord_i[1] == 0 and coord_j[0] == 0) or \
               (coord_i[1] != 0 and coord_j[0] != 0 and coord_i[0] == 0 and coord_j[1] == 0):

                if abs(time_i - time_j) < 10:

                    average_time = (time_i + time_j) / 2

                    coincidence_coord = [max(coord_i[0], coord_j[0]), max(coord_i[1], coord_j[1])]
                    
                    coincidences.append({
                        'event number': group.iloc[i]['event number'],
                        'rpc number': group.iloc[i]['rpc number'],
                        'coincidence coordinate': coincidence_coord,
                        'coincidence time': average_time
                    })
    return coincidences

In [129]:
print(df.groupby(['event number', 'rpc number']))

In [132]:
coincidences = []
for _, group in df.groupby(['event number', 'rpc number']):
    coincidences.extend(find_coincidences(group))

coincidences_df = pd.DataFrame(coincidences)

In [133]:
show(coincidences_df)

PandasGUI INFO — pandasgui.gui — Opening PandasGUI
INFO:pandasgui.gui:Opening PandasGUI


In [134]:
show(df)

PandasGUI INFO — pandasgui.gui — Opening PandasGUI
INFO:pandasgui.gui:Opening PandasGUI


Before I actually do this, I actually wanted to do something more on 1D with timing window first.... I will do that tmr

In [63]:
ni=unber

NameError: name 'unber' is not defined